In [2]:
from astropy.modeling.models import BlackBody1D
from astropy.modeling.blackbody import FLAM
from astropy import units as u
from astropy.visualization import quantity_support
from astropy.convolution import convolve_fft

import matplotlib as mpl
import matplotlib.pyplot as plt
from glob import glob
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact_manual
from ipywidgets import FloatSlider
from ipywidgets import Button, Layout, Box, HBox, VBox
from colorama import Fore
from IPython.display import display
from IPython.display import clear_output

import numpy as np
import pandas as pd
import math

mpl.rcParams['figure.figsize'] = 12, 9

In [3]:
sirius = pd.read_csv('Sirius.csv')
b_fil = pd.read_csv('B Filter.csv')
u_fil = pd.read_csv('U Filter.csv')
v_fil = pd.read_csv('V Filter.csv')

FileNotFoundError: File b'Sirius.csv' does not exist

In [ ]:
plt.plot(v_fil.iloc[:,0], v_fil.iloc[:,1])
plt.plot(b_fil.iloc[:,0], b_fil.iloc[:,1])
plt.plot(u_fil.iloc[:,0], u_fil.iloc[:,1])

In [ ]:
#Getting the UBV filter transmittance and wavelength data into arrays, and bandwidth data
i = 0
u_trans = np.array([])
for i in u_fil['Transmittance']:
    u_trans = np.append(u_trans, i)
i = 0
u_wav = np.array([])
for i in u_fil['lambda']:
    u_wav = np.append(u_wav, i)
b_trans = np.array([])
i = 0
for i in b_fil['Transmittance']:
    b_trans = np.append(b_trans,i)
i = 0
b_wav = np.array([])
for i in b_fil['lambda']:
    b_wav = np.append(b_wav, i)
v_trans = np.array([])
i = 0
for i in v_fil['Transmittance']:
    v_trans = np.append(v_trans, i)
i = 0
v_wav = np.array([])
for i in v_fil['lambda']:
    v_wav = np.append(v_wav, i)

u_band = u_wav[-1] - u_wav[0]
b_band = b_wav[-1] - b_wav[0]
v_band = v_wav[-1] - v_wav[0]
print('u', u_trans.size, u_wav.size)
print('b', b_trans.size, b_wav.size)
print('v', v_trans.size, v_wav.size)
print(u_band, b_band, v_band)

In [ ]:
#Getting approx. where Sirius' wavelength match the filters and append corresponding flux
#This only picks the first value within a threshold, not necessarily the closest value but its close enough for now
i = 0
j = 0
k = 0
l = 0
u_flux = np.array([])
b_flux = np.array([])
v_flux = np.array([])
while i < sirius.size:
    if sirius['lambda'][i] + 2 > u_wav[j] and sirius['lambda'][i] - 2 < u_wav[j]:
        u_flux = np.append(u_flux, sirius['Flux'][i])
        if j < u_wav.size - 1:
            j+=1
        else:
            j = 0
    if sirius['lambda'][i] + 3 > b_wav[k] and sirius['lambda'][i] - 3 < b_wav[k]:
        b_flux = np.append(b_flux, sirius['Flux'][i])
        if k < b_wav.size - 1:
            k+=1
        else:
            k = 0
    if sirius['lambda'][i] + 3 > v_wav[l] and sirius['lambda'][i] - 3 < v_wav[l]:
        v_flux = np.append(v_flux, sirius['Flux'][i])
        if l < v_wav.size - 1:
            l+=1
        else:
            l = 0
    if sirius['lambda'][i] > v_wav[-1]:
        break
    i+=1
print('u', u_flux.size)
print('b', b_flux.size)
print('v', v_flux.size)
print(u_flux)

In [ ]:
#Getting the light through the filter
def filt(u_trans, b_trans, v_trans, u_flux, b_flux, v_flux):
    i = 0
    fil_u = np.array([])
    fil_b = np.array([])
    fil_v = np.array([])
    while i < u_flux.size:
        temp_u = u_trans[i]*u_flux[i]
        fil_u = np.append(fil_u, temp_u)
        if i < b_flux.size - 1:
            temp_b = b_trans[i]*b_flux[i]
            fil_b = np.append(fil_b, temp_b)
        if i < v_flux.size - 1:
            temp_v = v_trans[i]*v_flux[i]
            fil_v = np.append(fil_v, temp_v)
        i+=1
    return fil_u, fil_b, fil_v
fil_u, fil_b, fil_v = filt(u_trans, b_trans, v_trans, u_flux, b_flux, v_flux)
print(fil_u)
print(fil_b)
print(fil_v)

In [ ]:
#Get U, B, V and Sirius correction
def ubv(fil_u, fil_b, fil_v):
    U = np.mean(fil_u) * u_band
    B = np.mean(fil_b) * b_band
    V = np.mean(fil_v) * v_band
    return U, B, V
U, B, V = ubv(fil_u, fil_b, fil_v)
def f(x,y):
    return -2.5*math.log10(x/y)
U_B = f(U, B)
B_V = f(B, V)
U_Bcorr = - U_B
B_Vcorr = - B_V

In [ ]:
#Plot and such for blackbody curve
h = 6.626e-34
c = 3e8
k = 1.38e-23
hc = h*c
def bb(wav, T):
    a = hc/(wav*k*T)
    p1 = 2*hc**2 / wav**5
    p2 = 1 / (np.exp(a) - 1)
    return p1*p2
def slider(T):
    wavelengths = np.arange(2e-9, 3e-6, 1e-9)
    i = bb(wavelengths, T)
    plt.plot(wavelengths*1e10, i, 'r-') 
    plt.ylabel('Intensity')
    plt.xlabel('Wavelength (Angstroms)')
    wavelengths *= 1e10
    wavelengths = np.round(wavelengths, 2)
    j = 0
    m = 0
    l = 0
    temp_u = np.array([])
    temp_b = np.array([])
    temp_v = np.array([])
    for temp in wavelengths:
        if temp == u_wav[j]:
            w = temp * 1e-10
            temp_u = np.append(temp_u, bb(w, T))
            if j < u_wav.size -1:
                j+=1
            else:
                j = 0
        if temp == b_wav[m]:
            w = temp * 1e-10
            temp_b = np.append(temp_b, bb(w, T))
            if m < b_wav.size -1:
                m+=1
            else:
                m = 0
        if temp == v_wav[l]:
            w = temp * 1e-10
            temp_v = np.append(temp_v, bb(w, T))
            if l < v_wav.size -1:
                l+=1
            else:
                l = 0
    tempfil_u, tempfil_b, tempfil_v = filt(u_trans, b_trans, v_trans, temp_u, temp_b, temp_v)
    temp_U, temp_B, temp_V = ubv(tempfil_u*1e10, tempfil_b*1e10, tempfil_v*1e10)
    tempU_B = f(temp_U, temp_B)
    tempB_V = f(temp_B, temp_V)
    lam_peak = 2.9e7/T
    print('Peak wavelength Angstroms:', lam_peak)
    print('U-B:', tempU_B + U_Bcorr)
    print('B-V:', tempB_V + B_Vcorr)
    return T
interact_manual(slider, T = 6000)

I know I did something wrong, but am not quite sure where. The B-V seems about right because it is about zero close to 10000K, but the U-B is close to zero at 5000K....